In [ ]:
# 测试读取数据

In [ ]:
# 数据同行运算

In [43]:
import pandas as pd

#风力数据
df = pd.DataFrame({'A': [40336, 2125, 3026, 107158, 630050, 800270, 70045, 900012, 600045]})
df['sped'] = df.apply(lambda x: (x.A - int(str(int(x.A))[-3:]))/10000, axis = 1)
df['dir'] = df.apply(lambda x: int(str(int(x.A))[-3:]), axis = 1)
df

,A,sped,dir
0,40336,4.0,336
1,2125,0.2,125
2,3026,0.3,26
3,107158,10.7,158
4,630050,63.0,50
5,800270,80.0,270
6,70045,7.0,45
7,900012,90.0,12
8,600045,60.0,45


In [52]:
# 行运算
import pandas as pd

def data(x):
    return x.A - x.B
#风力数据
df = pd.DataFrame({'A': [40336, 2125, 3026, 107158, 630050, 800270, 70045, 900012, 600045],
                  'B': [40336, 2125, 3026, 107158, 630050, 800270, 70045, 900012, 600045]})
df['sped'] = df.apply(data, axis = 1)
df

,A,B,sped
0,40336,40336,0
1,2125,2125,0
2,3026,3026,0
3,107158,107158,0
4,630050,630050,0
5,800270,800270,0
6,70045,70045,0
7,900012,900012,0
8,600045,600045,0


In [72]:
# 列运算
import pandas as pd

def data(x):
    #x.drop(3,inplace=True)
    print(x.name)
    x.loc[3] = 666
#     for i in range(len(x)):
#         if x[i] ==0:
#             x[i]=1
    da = sum(x)
    return da
#风力数据
df = pd.DataFrame({'A': [1, 2, 3, 0, 0, 0, 0, 0, 0],
                  'B': [1, 1, 1, 1, 1, 1, 1, 1, 1]})
df.apply(data, axis = 0).A


A
B


672

In [ ]:
# 小时数据更新

In [33]:
import datetime as dtt
now_times = "2023-07-18 13:15:00"
now = dtt.datetime.strptime(now_times,"%Y-%m-%d %H:%M:%S")
mins = now.minute
offset =dtt.timedelta(minutes=-mins)
hours = now + offset
 
start_time = hours.strftime('%Y-%m-%d %H:%M:%S')
end_time = now.strftime('%Y-%m-%d %H:%M:%S')

## 天擎

datetime.datetime(2023, 7, 18, 13, 0)

# 数据增量同步

In [53]:
%%time
import pandas as pd
data = pd.read_csv("downfile/min.csv")
data = data.replace(999999.0, np.nan)
sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','PRE','WIN_S_Gust_Max','WIN_D_Gust_Max','VIS_HOR_1MI','TEM']]
def comput_data(data,delay):
    grouped = data.groupby('Station_Id_C')
    for name,group in grouped:
        single = group
        # 单站处理
#         single['tmin'] = single['TEM']
#         single[single['WIN_S_Gust_Max']==999999]=-999999
#         max_time = single['Datetime'].max   
#         #station = pd.DataFrame({'current_time':single['Datetime'].max,'rain':sum(single['PRE']),'tmax':single['TEM'].max,'tmin':single['TEM'].min,'view':single['VIS_HOR_1MI'].min,'wind':single['VIS_HOR_1MI'].min})
#         # 计算聚合值
#     return single

delay = 24
single = comput_data(data,delay)
single


CPU times: user 344 ms, sys: 8.26 ms, total: 352 ms
Wall time: 364 ms


In [5]:
%%time
# 实时数据的统计
import numpy as np
import pandas as pd
data = pd.read_csv("downfile/min.csv")

data = data.replace(999999.0, np.nan)
data = data[((data['WIN_S_Gust_Max']>16.5)&(data['WIN_S_Gust_Max']<100.5)) |(((data['PRE']>0))&(data['PRE']>5000))]
grouped = data.groupby('Station_Id_C')
data_list = []
for name,group in grouped:
    single = group[['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','PRE','WIN_S_Gust_Max','WIN_D_Gust_Max']]
    wind = single['WIN_S_Gust_Max'].max()
    index =  data[data['WIN_S_Gust_Max'] == data['WIN_S_Gust_Max'].max()].index.tolist()[0]
    deg = data[data['WIN_S_Gust_Max'] == data['WIN_S_Gust_Max'].max()]['WIN_D_Gust_Max'].iloc[0]
    dit = {
        'Cnty':single['Cnty'].iloc[0],
        'Province':single['Province'].iloc[0],
        'Town':single['Town'].iloc[0],
        'Station_Name':single['Station_Name'].iloc[0],
        'Station_Id_C':single['Station_Id_C'].iloc[0],
        'Lat':single['Lat'].iloc[0],
        'Lon':single['Lon'].iloc[0],
        'Alti':single['Alti'].iloc[0],
        'rain':single['PRE'].sum,
        'wind':wind,
        'sped':deg
    }
    data_list.append(dit)




CPU times: user 1.6 s, sys: 7.52 ms, total: 1.61 s
Wall time: 1.62 s


In [45]:
data_list

'[1.0,1.7,0.4,4.5,0.6,0.1,0.1,4.0,0.1,0.4,0.3,0.2,2.7,0.3,0.3,0.1,0.3,0.4,0.1,3.3,0.1,0.3,2.4,0.3,0.4,0.1,0.2,0.8,0.4,2.8,1.3,0.7,0.2,1.2,0.7,0.8,2.6,1.9,0.5,0.4,0.1,3.4,0.1,1.4,0.4,0.1,2.2,1.7,0.1,0.5,1.0,1.4,0.3,0.6,0.2,0.2,0.2,0.2,3.4,1.7,0.5,0.2,5.0,0.4,0.1,0.9,0.2,0.1,0.1,0.3,0.5,0.6,0.5,0.3,0.1,0.4,0.7,1.4,0.8,0.5,1.0,0.1,0.1,0.8,0.1,0.3,3.0,1.4,1.3,1.3,0.1,0.5,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.2,0.1,0.1,0.1,0.2,0.2,0.1,0.1,0.2,0.4,0.2,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.4,0.1,0.3,0.2,0.1,0.1,0.6,0.1,2.2,0.4,0.1,0.1,0.1,0.8,0.8,1.3,1.3,0.2,3.1,0.6,1.0,0.3,3.3,0.1,0.5,0.1,0.1,0.1,0.1,0.1,0.5,0.1,0.1]'

In [1]:
%%time
# 实时数据的统计
import numpy as np
import pandas as pd
import json
data = pd.read_csv("downfile/min.csv")

data = data.replace(999999.0, np.nan)
wind = data[(data['WIN_S_Gust_Max']>0)&(data['WIN_S_Gust_Max']<5000)][['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','WIN_S_Gust_Max','WIN_D_Gust_Max']]

rain = data[(data['PRE']>0)&(data['PRE']<5000)][['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','PRE']]

wind_data = wind.groupby(['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','WIN_D_Gust_Max'])['WIN_S_Gust_Max'].max().reset_index().sort_values('WIN_S_Gust_Max', ascending=False).drop_duplicates(subset=['Station_Id_C'], keep='first').to_json(orient = "records", force_ascii=False)
rain_data = rain.groupby(['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti'])['PRE'].sum().reset_index().to_json(orient = "records", force_ascii=False)


/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


CPU times: user 791 ms, sys: 768 ms, total: 1.56 s
Wall time: 478 ms


In [20]:
wind_data = wind.groupby(['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','WIN_D_Gust_Max'])['WIN_S_Gust_Max'].max().reset_index()

In [24]:
wind_data.sort_values('WIN_S_Gust_Max', ascending=False).drop_duplicates(subset=['Station_Id_C'], keep='first')

# wind_data 

,Cnty,Province,Town,Station_Name,City,Station_Id_C,Lat,Lon,Alti,WIN_D_Gust_Max,WIN_S_Gust_Max
3941,嵊泗县,浙江省,嵊山镇,浪岗,舟山市,K9522,30.4403,122.9333,71.0,209.0,21.2
8926,瑞安市,浙江省,东山街道,北龙,温州市,K3074,27.6722,120.9764,51.0,243.0,17.7
5645,普陀区,浙江省,东极镇,东福山,舟山市,K9540,30.1344,122.7625,321.0,220.0,15.9
6792,椒江区,浙江省,大陈镇,一江山,台州市,K8116,28.6086,121.8156,50.0,238.0,15.7
8084,洞头区,浙江省,北岙街道,北岙三盘,温州市,K3486,27.8675,121.1642,37.0,228.0,15.3
...,...,...,...,...,...,...,...,...,...,...,...
6159,松阳县,浙江省,枫坪乡,枫坪乡中心小学,丽水市,K9080,28.3361,119.2656,424.0,95.0,0.1
2179,吴兴区,浙江省,滨湖街道,黄金湖岸,湖州市,K5199,30.9519,120.1214,3.0,346.0,0.1
3190,安吉县,浙江省,杭垓镇,桐杭,湖州市,K5104,30.5453,119.3642,131.0,297.0,0.1
2019,南湖区,浙江省,余新镇,幸福社区,嘉兴市,K5554,30.6842,120.8172,3.6,245.0,0.1


# 更新逐小时数据

In [ ]:
# 

In [29]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
import json
from datetime import timezone



class station_zdz:
    def __init__(self):
        self.rs = redis.Redis(host='127.0.0.1', port=6379)
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
    def sql_wind(self):
        start_times = '2023-07-18 12:00:00'
        end_times = '2023-07-18 12:59:00'
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province, max(Station_levl) as Station_levl,
        max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, max(WIN_S_Gust_Max*10000 + WIN_D_Gust_Max) as wind
        from table_station_min 
        where Datetime between '{start_times}' and '{end_times}' and WIN_S_Gust_Max<5000 and WIN_D_Gust_Max<5000
        group by Station_Id_C""" 
        rsql = sql.format(start_times=start_times,end_times=end_times)
        data = pd.read_sql(rsql, con=self.conn)
        sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','wind']]
        sql_data['Datetime'] = start_times
        datalist = sql_data.to_json(orient='values')
        sql_list = json.loads(datalist)
        conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        cursor = conn.cursor()
        insql = """insert into table_station_hour 
        (Station_Id_C,Cnty,City,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,wind,Datetime) 
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        on duplicate key update
        wind = values(wind),
        Station_Id_C = values(Station_Id_C),
        Cnty = values(Cnty),
        City = values(City),
        Province = values(Province),
        Station_levl = values(Station_levl),
        Station_Name = values(Station_Name),
        Town = values(Town),
        Alti = values(Alti),
        Station_Name = values(Station_Name),
        Lat = values(Lat),
        Lon = values(Lon),
        Datetime = values(Datetime)"""
        cursor.executemany(insql,sql_list)
        cursor.close()
        cursor = conn.cursor()
        conn.commit()
        conn.close()
    def sql_temp(self):
        start_times = '2023-07-18 12:00:00'
        end_times = '2023-07-18 12:59:00'
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province, max(Station_levl) as Station_levl,
        max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, max(TEM) as tmax,min(TEM) as tmin
        from table_station_min 
        where Datetime between '{start_times}' and '{end_times}' and TEM<5000 
        group by Station_Id_C""" 
        rsql = sql.format(start_times=start_times,end_times=end_times)
        data = pd.read_sql(rsql, con=self.conn)
        sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','tmax','tmin']]
        sql_data['Datetime'] = start_times
        datalist = sql_data.to_json(orient='values')
        sql_list = json.loads(datalist)
        conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        cursor = conn.cursor()
        insql = """insert into table_station_hour 
        (Station_Id_C,Cnty,City,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,tmax,tmin,Datetime) 
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        on duplicate key update
        tmax = values(tmax),
        tmin = values(tmin),
        Station_Id_C = values(Station_Id_C),
        Cnty = values(Cnty),
        City = values(City),
        Province = values(Province),
        Station_levl = values(Station_levl),
        Station_Name = values(Station_Name),
        Town = values(Town),
        Alti = values(Alti),
        Station_Name = values(Station_Name),
        Lat = values(Lat),
        Lon = values(Lon),
        Datetime = values(Datetime)"""
        cursor.executemany(insql,sql_list)
        cursor.close()
        cursor = conn.cursor()
        conn.commit()
        conn.close()
    def sql_rain(self):
        start_times = '2023-07-17 13:00:00'
        end_times = '2023-07-17 13:59:00'
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province, max(Station_levl) as Station_levl,
        max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, sum(PRE) as rain , max(PRE_1h) as rain_max
        from table_station_min 
        where Datetime between '{start_times}' and '{end_times}' and PRE<5000  
        group by Station_Id_C""" 
        rsql = sql.format(start_times=start_times,end_times=end_times)
        data = pd.read_sql(rsql, con=self.conn)
        sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','rain','rain_max']]
        sql_data['Datetime'] = start_times
        datalist = sql_data.to_json(orient='values')
        sql_list = json.loads(datalist)
        conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        cursor = conn.cursor()
        insql = """insert into table_station_hour 
        (Station_Id_C,Cnty,City,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,rain,rain_max,Datetime) 
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        on duplicate key update
        rain = values(rain),
        rain_max = values(rain_max),
        Station_Id_C = values(Station_Id_C),
        Cnty = values(Cnty),
        City = values(City),
        Province = values(Province),
        Station_levl = values(Station_levl),
        Station_Name = values(Station_Name),
        Town = values(Town),
        Alti = values(Alti),
        Station_Name = values(Station_Name),
        Lat = values(Lat),
        Lon = values(Lon),
        Datetime = values(Datetime)"""
        cursor.executemany(insql,sql_list)
        cursor.close()
        cursor = conn.cursor()
        conn.commit()
        conn.close()
    def sql_view(self):
        start_times = '2023-07-18 12:00:00'
        end_times = '2023-07-18 12:59:00'
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province,max(Station_levl) as Station_levl,
        max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, min(VIS_HOR_1MI) as view
        from table_station_min 
        where Datetime between '{start_times}' and '{end_times}' and VIS_HOR_1MI<30000 
        group by Station_Id_C""" 
        rsql = sql.format(start_times=start_times,end_times=end_times)
        data = pd.read_sql(rsql, con=self.conn)
        sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','view']]
        sql_data['Datetime'] = start_times
        datalist = sql_data.to_json(orient='values')
        sql_list = json.loads(datalist)
        conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        cursor = conn.cursor()
        insql = """insert into table_station_hour
        (Station_Id_C,Cnty,City,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,view,Datetime) 
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        on duplicate key update
        view = values(view),
        Station_Id_C = values(Station_Id_C),
        Cnty = values(Cnty),
        City = values(City),
        Province = values(Province),
        Station_levl = values(Station_levl),
        Station_Name = values(Station_Name),
        Town = values(Town),
        Alti = values(Alti),
        Station_Name = values(Station_Name),
        Lat = values(Lat),
        Lon = values(Lon),
        Datetime = values(Datetime)"""
        cursor.executemany(insql,sql_list)
        cursor.close()
        cursor = conn.cursor()
        conn.commit()
        conn.close()
    

    
worker = station_zdz()
data = worker.sql_rain()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 790 ms, sys: 10.2 ms, total: 800 ms
Wall time: 1.29 s


# 读取快报的内容

In [5]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
import json
from datetime import timezone
class station_plot:
    def __init__(self):
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        self.shp_path = "static/data/shpfile/country/"
    def get_sql_data(self,start,end):      
        '''sql 获取数据'''
        start = '2023-07-17 13:00:00'
        end = '2023-07-18 12:00:00'
        sqltall = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province,max(Station_levl) as Station_levl,
            max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, sum(rain) as PRE_sum,max(tmax) as tmax,min(tmin) as tmin ,max(wind)as wind ,min(view) as view
            from table_station_hour 
            where Datetime between '{start_times}' and '{end_times}' and City='台州市'
            group by Station_Id_C""" 
        rsql = sqltall.format(start_times=start,end_times=end)     
        data = pd.read_sql(rsql , con=self.conn)
        return data
start = '2023-07-17 13:00:00'
end = '2023-07-18 12:00:00'
worker = station_plot()
worker.get_sql_data(start,end)

CPU times: user 22.5 ms, sys: 1.54 ms, total: 24.1 ms
Wall time: 283 ms


,City,Cnty,Station_Id_C,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,PRE_sum,tmax,tmin,wind
0,台州市,天台县,58559,浙江省,12,天台,始丰街道,107.6,29.1528,120.9706,0.1,28.8,28.3,40120.0
1,台州市,三门县,58568,浙江省,12,三门,海游街道,34.5,29.1228,121.3833,4.3,27.2,27.0,26140.0
2,台州市,仙居县,58652,浙江省,12,仙居,安洲街道,83.2,28.8672,120.7189,0.0,29.6,28.9,27112.0
3,台州市,临海市,58653,浙江省,13,括苍山,括苍镇,1382.4,28.8097,120.9219,0.0,21.6,21.0,65201.0
4,台州市,临海市,58660,浙江省,12,临海,大洋街道,6.3,28.8739,121.1928,0.0,27.5,27.1,17185.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,台州市,临海市,KJ608,浙江省,14,红脚岩,桃渚镇,28.0,28.8322,121.6825,0.0,29.0,28.7,28220.0
577,台州市,临海市,KJ609,浙江省,14,马里岙,涌泉镇,138.0,28.7783,121.3394,4.7,25.5,25.4,NaN
578,台州市,临海市,KJ611,浙江省,14,跑马坪,括苍镇,1255.0,28.7853,120.9142,0.0,21.5,21.3,47251.0
579,台州市,临海市,KJ617,浙江省,14,永丰中心校,永丰镇,17.0,28.8800,121.0403,0.0,28.6,28.1,29077.0


In [ ]:
# 